### Purpose

Make MESA directories investigating the seismic signatures of MS stars that undergo mass transfer. (gain or loss)

In [1]:
from qol.mesa.launcher import *
import qol.info as info

#### TODO

check out history_columns.list

In [ ]:
# Presets
mass_initial = 1.
mass_final = 1.5
Xcen_accrete = 0.5 # core hydrogen fraction at accretion time
Mdot_accrete = 1e-5
overshoot_f = 0.015
overshoot_f0 = 0.005

In [ ]:
# EVOLUTION BEFORE MASS TRANSFER

inlist = MesaInlist('inlist_pre_mass_transfer', LOGS_dir='LOGS/pre_mass_transfer/')
inlist.enable_pgstar()
inlist.save_pgstar(write_path='Grid1/pre_mass_transfer/')
inlist.use_qol_pgstar()

# Write GYRE
inlist.write_gyre_data_with_profile()

# Initial parameters
inlist.initial_mass(mass_initial)
inlist.set_Zbase(0.02)

# Termination conditions
inlist.stop_at_phase_TAMS()
inlist.add_xa_central_lower_limit(xa_central_lower_limit_species='h1',
                                  xa_central_lower_limit=Xcen_accrete) # stop this evolution when core drops to desired hydrogen fraction
inlist.save_final_model('pre_mass_transfer.mod')

# Overshoot
inlist.add_overshoot_zone(overshoot_scheme='exponential',
                          overshoot_zone_type='any', overshoot_zone_loc='any', overshoot_bdy_loc='any',
                          overshoot_f=overshoot_f, overshoot_f0=overshoot_f0)

task_pre_mass_transfer = inlist

In [ ]:
# EVOLUTION THROUGH MASS TRANSFER AND POST MASS TRANSFER

inlist = MesaInlist('inlist_to_tams', LOGS_dir='LOGS/to_tams/')
inlist.enable_pgstar()
inlist.save_pgstar(write_path='Grid1/to_tams/')
inlist.use_qol_pgstar()

# Initial conditions
inlist.load_model('pre_mass_transfer.mod')
inlist.set_Zbase(0.02)

# Write GYRE
inlist.write_gyre_data_with_profile()

# Mass transfer
if mass_final > mass_initial:
    inlist.gain_mass(max_star_mass_for_gain=mass_final, mass_change=Mdot_accrete)
elif mass_final < mass_initial:
    inlist.lose_mass(min_star_mass_for_loss=mass_final, mass_change=Mdot_accrete)

# Termination conditions
inlist.stop_at_phase_TAMS()
inlist.save_final_model('to_tams.mod')

# Overshoot
inlist.add_overshoot_zone(overshoot_scheme='exponential',
                          overshoot_zone_type='any', overshoot_zone_loc='any', overshoot_bdy_loc='any',
                          overshoot_f=overshoot_f, overshoot_f0=overshoot_f0)

task_to_zams = inlist

In [ ]:
# Put it together

work = MesaWorkingDirectory(run_path='/Users/nrui/Desktop/mtglitch_test/')
work.copy_history_columns_list(f'{info.qol_path}mesa/resources/r24.08.1/history_columns.list')
work.copy_profile_columns_list(f'{info.qol_path}mesa/resources/r24.08.1/profile_columns.list')
work.load_qol_pgstar()

work.add_task(task_pre_mass_transfer)
work.add_task(task_to_zams)

work.save_directory(grant_perms=True, source_sdk=True)